In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [180]:
fmm_fft_df_raw = pd.read_csv('data_fmmfft.txt', names = ['N', 'p', 'b', 't', 'time', 'err']).drop(columns=['b','t', 'err'])
serial_df_raw = pd.read_csv('data_serial.txt', names = ['N', 'run_num', 'time']).drop(columns=['run_num'])
fftw_mpi_df_raw = pd.read_csv('out_fftw_mpi.txt', usecols=[0,1,2,3],names = ['N', 'p', 'run_num','time']).drop(columns='run_num')

In [202]:
serial_df = serial_df_raw.groupby(['N'], as_index=False).max()
fftw_mpi_df = fftw_mpi_df_raw.groupby(['N', 'p'], as_index=False).min()
fmm_fft_df = fmm_fft_df_raw.groupby(['N', 'p'], as_index=False).min()

## Speedup w.r.t Serial

In [203]:
problem_sizes = sorted(list(set(serial_df['N'].unique().tolist()) & set(fmm_fft_df['N'].unique().tolist())))
cores = sorted(fmm_fft_df['p'].unique().tolist())

speedup_serial_df_matrix = []

for n in problem_sizes:
    row_insert = [n]
    for p in cores:
        #print(n,p)
        fmm_fft_rows = fmm_fft_df.loc[(fmm_fft_df['N']==n) & (fmm_fft_df['p']==p)]
        serial_rows = serial_df.loc[(serial_df['N']==n)]
        if fmm_fft_rows.shape[0] == 0 or serial_rows.shape[0]==0: 
            row_insert.append(0)
            continue 
        fmm_fft_time = fmm_fft_rows.iloc[0]['time']
        serial_time = serial_rows.iloc[0]['time']
        speedup = serial_time/fmm_fft_time
        row_insert.append(speedup)
    speedup_serial_df_matrix.append(row_insert)

speedup_serial_df = pd.DataFrame(speedup_serial_df_matrix, columns=['N'] + [str(c) for c in cores])
speedup_serial_df

,N,2,4,8,16,32,64
0,32768,0.250591,0.212657,0.158016,0.063521,0.030330,0.000000
1,65536,0.281882,0.255792,0.269918,0.077180,0.049526,0.000000
2,131072,0.286432,0.290483,0.353754,0.017115,0.101261,0.009533
3,262144,0.616593,0.642769,0.855923,0.344820,0.034425,0.025919
4,524288,0.355155,0.417882,0.606950,0.217381,0.021973,0.041130
5,1048576,0.405789,0.521740,0.820543,0.483790,0.064526,0.024493
6,2097152,0.381067,0.469399,0.815457,0.560860,0.080341,0.039684
7,4194304,0.450917,0.560381,0.924900,0.884275,0.160300,0.090005
8,8388608,0.457960,0.543935,0.936123,0.777923,0.238165,0.146946
9,16777216,0.464336,0.551699,0.940766,0.825549,0.458002,0.203803


## Speedup w.r.t vanilla Distributed FFT

In [204]:
problem_sizes = sorted(list(set(fftw_mpi_df['N'].unique().tolist()) & set(fmm_fft_df['N'].unique().tolist())))
cores = sorted(list(set(fftw_mpi_df['p'].unique().tolist()) & set(fmm_fft_df['p'].unique().tolist())))

speedup_fftw_df_matrix = []


for n in problem_sizes:
    row_insert = [n]
    for p in cores:
        fmm_fft_rows = fmm_fft_df.loc[(fmm_fft_df['N']==n) & (fmm_fft_df['p']==p)]
        fftw_rows = fftw_mpi_df.loc[(fftw_mpi_df['N']==n) & (fftw_mpi_df['p']==p)]
        if fmm_fft_rows.shape[0] == 0 or fftw_rows.shape[0]==0: 
            row_insert.append(0)
            continue 
        fmm_fft_time = fmm_fft_rows.iloc[0]['time']
        fftw_time = fftw_rows.iloc[0]['time']
        speedup = fftw_time/fmm_fft_time
        row_insert.append(speedup)
    speedup_fftw_df_matrix.append(row_insert)

pd.DataFrame(speedup_fftw_df_matrix, columns=['N'] + [str(c) for c in cores])



,N,2,4,8,16,32,64
0,32768,1.175156,0.792449,0.766635,1.073328,0.552414,0.000000
1,65536,1.347330,1.530754,1.113304,7.904475,0.535864,0.000000
2,131072,1.141842,1.466281,1.096200,0.355704,0.959121,0.097060
3,262144,1.713492,1.527426,1.672239,2.004559,2.627005,1.014840
4,524288,1.218792,1.724951,1.710506,7.931457,1.492384,1.878936
5,1048576,1.137769,1.570730,1.766113,2.184993,2.601403,2.071869
6,2097152,1.112610,1.074902,1.670780,2.340369,2.272808,1.725931
7,4194304,1.076480,1.408438,1.523904,3.226894,0.966694,2.030165
8,8388608,1.084940,0.890980,1.493563,2.190356,1.213090,1.708397
9,16777216,1.066901,0.863970,1.393347,2.130120,1.893733,0.968251


In [190]:
fmm_fft_df.loc[(fmm_fft_df['N']==1073741824) & (fmm_fft_df['p']==64)]

,N,p,time
87,1073741824,64,389.856143


In [193]:
fftw_mpi_df_raw.loc[(fftw_mpi_df_raw['N']==1073741824) & (fftw_mpi_df_raw['p']==64)]

,N,p,time
455,1073741824,64,450.462903
456,1073741824,64,302.060156
457,1073741824,64,611.578536
458,1073741824,64,1263.341479
459,1073741824,64,88.609889
